In [10]:
import pandas as pd
import pandas_ta as ta
import pandas as pd
import numpy as np

import datetime
import MetaTrader5 as mt5
%matplotlib inline
import talib

from scipy.signal import savgol_filter
from scipy.signal import find_peaks

pd.set_option('mode.chained_assignment', None)

In [17]:
if not mt5.initialize(
    login=114999529, server="Exness-MT5Trial6", password="Mypassword$1234"
):
    print("initialize() failed, error code =", mt5.last_error())
    quit()

login = 114999529
password = "Mypassword$1234"
server = "Exness-MT5Trial6"
mt5.login(login, password, server)
rate = mt5.copy_rates_from('EURUSD', mt5.TIMEFRAME_M15, datetime.datetime.now(), 1000)
df = pd.DataFrame(rate)

df['date']=pd.to_datetime(df['time'], unit='s')

df.set_index(np.arange(len(df)), inplace = True)
df = df.drop(['spread', 'real_volume', 'tick_volume', 'time'], axis='columns')

# df['RSI'] = ta.rsi(df.close, length=12)
# df['EMA'] = ta.ema(df.close, length=150)


# df=df[-5000:]

df.set_index(np.arange(len(df)), inplace = True)

df['atr'] = ta.atr(high=df.high, low=df.low, close=df.close)

df['atr'] = df.atr.rolling(window=30).mean()

In [18]:
df["close_smooth"] = savgol_filter(df.close, 30, 10)

atr = df.atr.iloc[-1]

peaks_idx, _ = find_peaks(df.close_smooth, distance=5, width=1, prominence=atr/10)

print(df.close_smooth.iloc[peaks_idx[-1]])

# print(peaks_idx[-1])

troughs_idx, _ = find_peaks(-1 * df.close_smooth, distance=5, width=1, prominence=atr/10)

1.0882161071234722


In [19]:
# Find index

def find_index(arr, input_value):
    low, high = 0, len(arr) - 1
    index = None

    while low <= high:
        mid = (low + high) // 2

        if arr[mid] < input_value:
            index = mid
            low = mid + 1
        else:
            high = mid - 1

    return index

In [20]:
# EMA

df["EMA_200"] = talib.EMA(df["close"], timeperiod=200)

In [21]:
# print(peaks_idx)

def trend(row):
    # Identify the Trend
    global df, peaks_idx, troughs_idx
    
    # Index of candle in DF 
    df_index = df[df["date"] == row["date"]].index[0]
    peak_index = find_index(peaks_idx, df_index)
    trough_index= find_index(troughs_idx, df_index)
    # print('df index: ', df_index)
    # print('Peak index: ',peak_index)
    # print('Peak Value: ', peaks_idx[peak_index])
    # print('Trough index: ',trough_index)
    # print('Peak Value: ', troughs_idx[trough_index])

    if len(peaks_idx) > 2 and len(troughs_idx) > 2:
        
        if (
            # Current Price > EMA 
            row['close'] > row['EMA_200'] and 
            # Current price > Current Low
            df.loc[troughs_idx[trough_index], 'close'] <= row['close'] and
            # Current Low > Prev Low
            df.loc[troughs_idx[trough_index-1], 'close'] <= df.loc[troughs_idx[trough_index], 'close'] and
            # Current High > Prev High
            df.loc[peaks_idx[peak_index], 'close'] > df.loc[peaks_idx[peak_index-1], 'close']
        ):
            df.loc[df_index, 'trend'] = 'up'

        elif (
            # Current Price < EMA
            row['close'] < row['EMA_200'] and
            # Current price < Current High
            row['close'] <= df.loc[peaks_idx[peak_index], 'close'] and
            # Current High < Prev High
            df.loc[peaks_idx[peak_index], 'close'] < df.loc[peaks_idx[peak_index-1], 'close'] and
            # Current Low < Prev Low
            df.loc[troughs_idx[trough_index], 'close'] < df.loc[troughs_idx[trough_index-1], 'close']
        ):
            df.loc[df_index, 'trend'] = 'down'

        
        else:

            df.loc[df_index, 'trend'] = 'no trend'


    

In [22]:
df.apply(trend, axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
995    None
996    None
997    None
998    None
999    None
Length: 1000, dtype: object

In [ ]:
# Small Savgol

small_peaks_idx,_ = find_peaks(df.close_smooth, distance = 10, width = 2, prominence= atr/10)

small_troughs_idx,_ = find_peaks(-1*df.close_smooth, distance = 10, width = 2, prominence=atr/10)

In [9]:
df.to_excel('EURUSD.xlsx')